In [181]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [182]:
news_df = pd.read_csv('train.csv', encoding='ISO-8859-1')


<ipython-input-182-7a4d8d8687b4>:1: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,2

In [183]:
news_df.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 676,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
# Count of each label
print(news_df['label'].value_counts())


label
0    518
1    506
Name: count, dtype: int64


In [185]:
news_df = news_df[news_df['label'].isin([0, 1])]

In [139]:
news_df = news_df[['id', 'title', 'author', 'text', 'label']]



In [188]:
news_df.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 676,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685
18432,14836,CLOWN PORN SEARCHES UP 213%,Iron Sheik,Home âº SOCIETY âº CLOWN PORN SEARCHES UP 21...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18433,14837,Krikorian: âBringing Sanctuary Cities to Hee...,Dan Riehl,"Mark Krikorian, the executive director of the ...",0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18434,14838,Spain Arrests Islamic State Extremists Plottin...,Breitbart London,MADRID (AP) â Spainâs Interior Ministry...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18435,14839,"Italy to Speed up Migrant Asylum Decisions, De...",Breitbart London,ROME (AP) â The Italian government on Fri...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18436,14840,âLast Night in Swedenâ? Trumpâs Remark B...,Sewell Chan,"LONDON â Swedes reacted with confusion, a...",0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
news_df.shape

(1024, 686)

In [190]:
news_df.isna().sum()

,0
id,0
title,20
author,113
text,1
label,0
...,...
Unnamed: 681,1024
Unnamed: 682,1024
Unnamed: 683,1024
Unnamed: 684,1024


In [191]:
news_df.isna().sum()

,0
id,0
title,20
author,113
text,1
label,0
...,...
Unnamed: 681,1024
Unnamed: 682,1024
Unnamed: 683,1024
Unnamed: 684,1024


In [192]:
news_df=news_df.fillna(' ')

<ipython-input-192-82b5457a134d>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  news_df=news_df.fillna(' ')


In [193]:
news_df.isna().sum()

,0
id,0
title,0
author,0
text,0
label,0
...,...
Unnamed: 681,0
Unnamed: 682,0
Unnamed: 683,0
Unnamed: 684,0


In [194]:
news_df.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 676,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685
18432,14836,CLOWN PORN SEARCHES UP 213%,Iron Sheik,Home âº SOCIETY âº CLOWN PORN SEARCHES UP 21...,1,,,,,,...,,,,,,,,,,
18433,14837,Krikorian: âBringing Sanctuary Cities to Hee...,Dan Riehl,"Mark Krikorian, the executive director of the ...",0,,,,,,...,,,,,,,,,,
18434,14838,Spain Arrests Islamic State Extremists Plottin...,Breitbart London,MADRID (AP) â Spainâs Interior Ministry...,0,,,,,,...,,,,,,,,,,
18435,14839,"Italy to Speed up Migrant Asylum Decisions, De...",Breitbart London,ROME (AP) â The Italian government on Fri...,0,,,,,,...,,,,,,,,,,
18436,14840,âLast Night in Swedenâ? Trumpâs Remark B...,Sewell Chan,"LONDON â Swedes reacted with confusion, a...",0,,,,,,...,,,,,,,,,,


In [195]:
news_df['content']=news_df['author']+' '+news_df['title']

<ipython-input-195-00d9f09682fd>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  news_df['content']=news_df['author']+' '+news_df['title']


In [196]:
news_df.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685,content
18432,14836,CLOWN PORN SEARCHES UP 213%,Iron Sheik,Home âº SOCIETY âº CLOWN PORN SEARCHES UP 21...,1,,,,,,...,,,,,,,,,,Iron Sheik CLOWN PORN SEARCHES UP 213%
18433,14837,Krikorian: âBringing Sanctuary Cities to Hee...,Dan Riehl,"Mark Krikorian, the executive director of the ...",0,,,,,,...,,,,,,,,,,Dan Riehl Krikorian: âBringing Sanctuary Cit...
18434,14838,Spain Arrests Islamic State Extremists Plottin...,Breitbart London,MADRID (AP) â Spainâs Interior Ministry...,0,,,,,,...,,,,,,,,,,Breitbart London Spain Arrests Islamic State E...
18435,14839,"Italy to Speed up Migrant Asylum Decisions, De...",Breitbart London,ROME (AP) â The Italian government on Fri...,0,,,,,,...,,,,,,,,,,Breitbart London Italy to Speed up Migrant Asy...
18436,14840,âLast Night in Swedenâ? Trumpâs Remark B...,Sewell Chan,"LONDON â Swedes reacted with confusion, a...",0,,,,,,...,,,,,,,,,,Sewell Chan âLast Night in Swedenâ? Trumpâ...


In [197]:
import nltk

In [198]:
!pip install nltk

In [199]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [200]:
import string

In [201]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [202]:
def transform_text(text):
    text=text.lower()
    text=nltk.word_tokenize(text)

    y=[]
    for i in text:
      if i.isalnum():
        y.append(i)
    text=y[:]
    y.clear()

    for i in text:
      if i not in stopwords.words('english') and i not in string.punctuation:
        y.append(i)
    text=y[:]
    y.clear()

    for i in text:
      y.append(ps.stem(i))

    return " ".join(y)

In [203]:
transform_text('I loved the YT lectures on Machine Learning')

'love yt lectur machin learn'

In [204]:
news_df['transformed_text']=news_df['content'].apply(transform_text)

<ipython-input-204-38183d5d0d9e>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  news_df['transformed_text']=news_df['content'].apply(transform_text)


In [205]:
news_df.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685,content,transformed_text
18432,14836,CLOWN PORN SEARCHES UP 213%,Iron Sheik,Home âº SOCIETY âº CLOWN PORN SEARCHES UP 21...,1,,,,,,...,,,,,,,,,Iron Sheik CLOWN PORN SEARCHES UP 213%,iron sheik clown porn search 213
18433,14837,Krikorian: âBringing Sanctuary Cities to Hee...,Dan Riehl,"Mark Krikorian, the executive director of the ...",0,,,,,,...,,,,,,,,,Dan Riehl Krikorian: âBringing Sanctuary Cit...,dan riehl krikorian sanctuari citi heel much b...
18434,14838,Spain Arrests Islamic State Extremists Plottin...,Breitbart London,MADRID (AP) â Spainâs Interior Ministry...,0,,,,,,...,,,,,,,,,Breitbart London Spain Arrests Islamic State E...,breitbart london spain arrest islam state extr...
18435,14839,"Italy to Speed up Migrant Asylum Decisions, De...",Breitbart London,ROME (AP) â The Italian government on Fri...,0,,,,,,...,,,,,,,,,Breitbart London Italy to Speed up Migrant Asy...,breitbart london itali speed migrant asylum de...
18436,14840,âLast Night in Swedenâ? Trumpâs Remark B...,Sewell Chan,"LONDON â Swedes reacted with confusion, a...",0,,,,,,...,,,,,,,,,Sewell Chan âLast Night in Swedenâ? Trumpâ...,sewel chan night remark baffl nation new york ...


In [206]:
for msg in news_df[news_df['label'] == 1]['transformed_text'].head(10):
    print(msg)


iron sheik clown porn search 213
guy made 84 watt death star home
daisi luther trump protest feed clinton support take street team win
crazi talk bori johnson britain arm saudi other happili
pakalert joel skousen scenario world war 3 nwo 2016 come
v saxena hillari help friend visa issu within 2 hour
lindsay hayward assang confirm depart justic told hillari delet evid conserv daili post
patrick slatteri nation bugl radio first step toward overthrow jewish power
mari lyon describ
norepli der postillon ratgeb all sie jetzt ã¼ber die wissen mã¼ssen


In [207]:
fake_corpus = []

for msg in news_df[news_df['label'] == 1]['transformed_text'].tolist():
    for word in msg.split():
        fake_corpus.append(word)


In [208]:
len(fake_corpus)

4158

In [209]:
real_corpus=[]
for msg in news_df[news_df['label']==0]['transformed_text'].tolist():
  for word in msg.split():
    real_corpus.append(word)

In [210]:
len(real_corpus)

5703

In [211]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv=CountVectorizer()
tfidf=TfidfVectorizer(max_features=3000)


In [212]:
X=tfidf.fit_transform(news_df['transformed_text']).toarray()

In [213]:
y=news_df['label'].values

In [214]:
from sklearn.model_selection import train_test_split

In [215]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)


In [216]:
print(set(y_train))
print(set(map(type, y_train)))  # Should show only <class 'int'> or <class 'numpy.int64'>


{np.int64(0), np.int64(1)}
{<class 'numpy.int64'>}


In [217]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [222]:
from sklearn.ensemble import RandomForestClassifier

modelR = RandomForestClassifier()
modelR.fit(X_train, y_train)


RandomForestClassifier()

In [218]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [223]:
modelR.fit(X_train,y_train)
y_pred1=model.predict(X_test)
print(accuracy_score(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))
print(precision_score(y_test,y_pred1))

0.9317073170731708
[[ 91  13]
 [  1 100]]
0.8849557522123894


In [225]:
import requests
import json  # add this

api_key = '8c621621b03d41d78713ae263c5d0e1e'  # Replace if needed
url = "https://newsapi.org/v2/top-headlines"
params = {
    'country': 'us',  # change to 'us'
    'category': 'general',
    'pageSize': 10,
    'apiKey': api_key
}
response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
    articles = data['articles']
    titles = [article['title'] for article in articles if article['title'] is not None]

    print("\n🔍 Live News Classification Results:\n")
    for title in titles:
        transformed = transform_text(title)
        vectorized = tfidf.transform([transformed])
        proba = model.predict_proba(vectorized)[0]
        pred = model.predict(vectorized)[0]
        print(f"REAL: {proba[0]:.2f}, FAKE: {proba[1]:.2f}")
        label = "FAKE ❌" if pred == 1 else "REAL ✅"
        print(f"{label} → {title}")
else:
    print("Failed to fetch news. Status Code:", response.status_code)



🔍 Live News Classification Results:

REAL: 0.19, FAKE: 0.81
FAKE ❌ → Trump tariffs: Officials vow to stay course as countries scramble - BBC
REAL: 0.30, FAKE: 0.70
FAKE ❌ → Yemen's Houthis say US strikes kill 3 as Trump's bombing video suggests higher overall death toll - AP News
REAL: 0.02, FAKE: 0.98
FAKE ❌ → Pirates apologize, plan to re-add Clemente sign - ESPN
REAL: 0.20, FAKE: 0.80
FAKE ❌ → Cantwell: GOP support for limit on tariffs shows ‘the anxiety that people have’ - Politico
REAL: 0.07, FAKE: 0.93
FAKE ❌ → Alex Ovechkin breaks Wayne Gretzky’s NHL all-time career goals record - CNN
REAL: 0.09, FAKE: 0.91
FAKE ❌ → South Carolina-UConn: Score, live updates for the 2025 women's NCAA title game - NCAA.com
REAL: 0.12, FAKE: 0.88
FAKE ❌ → Tesla Bull Slashes Stock Price Target 43%, Citing Musk and Trump - Bloomberg.com
REAL: 0.26, FAKE: 0.74
FAKE ❌ → Judge again orders Trump administration to return man mistakenly deported to an El Salvador prison - AP News
REAL: 0.07, FAKE: 0.93
F